<a href="https://colab.research.google.com/github/MuajiiTsai/110-1-NTU-DBME5028/blob/raw_label/DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn import metrics
from PIL import Image
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from google.colab import drive

model_path = "/content/drive/My Drive/Deep_Learning/Midterm/model.pt"

##Connect Google Drive

In [ ]:
drive.mount('/content/drive')
train_path = '/content/drive/My Drive/Deep_Learning/Midterm/train/'
os.chdir(train_path)
training_data_file = os.listdir(train_path)
print(training_data_file[0][:-11])
type(training_data_file)
# os.chdir(test_path)
# testing_data_file = os.listdir(test_path)
# print(len(training_data_file))

Mounted at /content/drive
WRIST_dad441e1e1


list

##Dataset

In [ ]:
from torchvision.io import read_image, ImageReadMode
#train_dataset

Resize = transforms.Compose([
  transforms.Resize([512,512])
])

class BoneDataset(Dataset):
    def __init__(self, annotations_file, directory, transform=None, target_transform=None):
        self.dir = directory
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = os.listdir(directory)
        self.transform = transform
        self.target_transform = target_transform
        self.classification()
    def __len__(self):
        return len(self.img_dir)
    def __getitem__(self, idx):
        img_path = self.img_dir[idx]
        image = read_image(os.path.join(self.dir, img_path), mode=ImageReadMode.RGB)
        image = image.type(torch.FloatTensor)
        label = self.getlabel(img_path)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    def classification(self):
        # 
        # self.img_labels = self.img_labels.sort_values(by=['label'])
        # print(self.img_labels)
        # 
        self.img_labels.set_index('label', inplace=True)
        # print(self.img_labels[0.0])
        labels_nan = self.img_labels.loc[np.nan]
        self.img_labels.reset_index(inplace=True)
        labels_nan.reset_index(inplace=True)
        # 
        # self.img_dir = self.img_dir[labels_nan[0]:]
        # 
        # data_with_label = []    #some func
        
    def getlabel(self, img_path):
        self.img_labels.set_index('id', inplace=True)
        i = self.img_labels.loc[f"{img_path[:-11]}"]
        label = i['label']
        self.img_labels.reset_index(inplace=True)
        return label

In [ ]:
train_path = '/content/drive/My Drive/Deep_Learning/Midterm/train/'
test_path = '/content/drive/My Drive/Deep_Learning/Midterm/test/'
label_file = '/content/drive/My Drive/Deep_Learning/Midterm/train.csv'

#model
batch_size = 4
learning_rate = 1e-4
num_show = 3
transform = transforms.Compose([
  transforms.Grayscale(num_output_channels=3),
  # transforms.PILToTensor(),
  transforms.Resize([512,512]),
  # transforms.ToPILImage(mode='RGB')
])



train_dataset = BoneDataset(label_file, train_path, transform=transform)
# test_dataset = ...


train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle = True
)
'''
test_dataloader = torch.utils.data.DataLoader(
    #test_dataset,
    batch_size=1,
    shuffle = False
)
'''
model = models.efficientnet_b0(pretrained=True)
# model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(
    1,
    64,
    kernel_size=(7, 7),
    stride=(2, 2),
    padding=(3, 3),
    bias=False
)
model = nn.Sequential(model, nn.Linear(1000,1))
# model = model.cuda()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = nn.SmoothL1Loss()
# print(train_dataloader.dataset)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

In [ ]:
train_features, train_labels = next(iter(train_dataloader))
print(f"feature batch size: {train_features.size()}")
print(f"labels batch size: {train_labels.size()}")
img = train_features[1].squeeze()
label = train_labels[1]
# print(img)
# plt.imshow(img)
# plt.show()
print(f"Label: {label}")

feature batch size: torch.Size([4, 3, 512, 512])
labels batch size: torch.Size([4])
Label: nan


In [ ]:
num_epochs = 1

#train the model
best_test_loss = np.inf

for idx_epoch in range(num_epochs):
  print(f"Epoch{idx_epoch}")

  #training phase
  model.train()

  train_losses = []
  for image_batch, label_batch in tqdm(train_dataloader):
    optimizer.zero_grad()
    image_batch = image_batch.cuda()
    label_batch = label_batch.cuda()
    output_batch = model(image_batch)
    # print(output_batch.shape)
    loss = loss_fn(output_batch, label_batch)
    loss.backward()
    optimizer.step()
    
    loss = loss.detach().cpu().numpy()
    train_losses.append(loss)

  train_loss = np.mean(train_losses)
  print(f"Training loss: {train_loss}")

  # y = np.array([1,1,2,2])
  # y_pred = np.array([0.1,0.4,0.35,2])
  # fpr, tpr, thresholds = metrics.roc_curve(y, y_pred, pos_label=2)
  # print(metrics.auc(fpr, tpr))

  '''
  #testing phase
  model.eval()

  test_losses = []
  test_accuracies = []
  for image_batch, label_batch in tqdm(test_dataloader):
    image_batch = image_batch.cuda()
    label_batch = label_batch.cuda()
    output_batch = model(image_batch)
    loss = loss_fn(output_batch, label_batch)
    
    loss = loss.detach().cpu().numpy()
    train_losses.append(loss)
    
    ##### AUC #####
    

    accuracy = np.mean(is_correct_batch)
    test_accuracies.append(accuracy)
    
    

    #####
    test_loss = np.mean(test_losses)
    test_accuracy = np.mean(test_accuracies)

    if test_loss < best_test_loss:
      best_test_loss = test_loss
      torch.save(model.state_dict(), model_path)
      print("The model is saved")
    '''
  torch.save(model.state_dict(), model_path)
  print("The model is saved")

Epoch0


  0%|          | 0/2685 [00:00<?, ?it/s]

NameError: ignored

#Playground

##Import CSV

In [ ]:
df = pd.read_csv (r'/content/drive/My Drive/Deep_Learning/Midterm/train.csv')
print(df)
# sorted_df = df.sort_values(by="label",kind='mergesort')
# i = df.loc["FOREARM_00f72d1153"]
df.set_index('label', inplace=True)
print(df)
j = df.loc[np.nan]
j.reset_index(inplace=True)
df.reset_index(inplace=True)
print(j)
print(j['id'][0])
# print(i['label'])
# print(j)
# print(df)
# print(sorted_df)
# df.index

                      id  label
0     FOREARM_000b20cbe3    NaN
1     FOREARM_000be2e9ab    1.0
2     FOREARM_00192d1b21    NaN
3     FOREARM_0055c4c62a    1.0
4     FOREARM_00f72d1153    0.0
...                  ...    ...
3995    WRIST_ff39824aa4    0.0
3996    WRIST_ff5128afcd    1.0
3997    WRIST_ff60656c01    0.0
3998    WRIST_ffaf60a955    0.0
3999    WRIST_fff3b05bc0    0.0

[4000 rows x 2 columns]
                       id
label                    
NaN    FOREARM_000b20cbe3
1.0    FOREARM_000be2e9ab
NaN    FOREARM_00192d1b21
1.0    FOREARM_0055c4c62a
0.0    FOREARM_00f72d1153
...                   ...
0.0      WRIST_ff39824aa4
1.0      WRIST_ff5128afcd
0.0      WRIST_ff60656c01
0.0      WRIST_ffaf60a955
0.0      WRIST_fff3b05bc0

[4000 rows x 1 columns]
      label                  id
0       NaN  FOREARM_000b20cbe3
1       NaN  FOREARM_00192d1b21
2       NaN  FOREARM_012cefba64
3       NaN  FOREARM_01eb7f88b4
4       NaN  FOREARM_02cc236db2
...     ...                 ...
1345

##Import Data

In [ ]:
train_dataset = torch.tensor([])
transform1 = transforms.Compose([
  transforms.PILToTensor(),
])
print(train_dataset)
for i in range(100):
  img = Image.open(f"{train_path}/{training_data_file[i]}")
  img_tensor = transform1(img)
  a = torch.cat((train_dataset, img_tensor),0)
  # print(a.shape)

tensor([])


KeyboardInterrupt: ignored